- Qobj would be the first thing and important to start using QuTiP
- The submodule of QIP, for gates, circuits, tomography and visualization
- End off with a fancy demonstration of QIP's pulse level optimization just to illustrate what's possible.



* First hour:
  * Introduction [5 minutes]
  * Overview [5 minutes]
    * What we are going to cover during the tutorial
  * Opening Python and importing QuTiP [5 minutes]
    * What is QuTiP?

* Second hour:
  * Understanding the evolution of quantum systems [20 minutes]
    * Understanding the Schrodinger equation
    * Understanding the role of the Hamiltonian and the energy of a system
    * Understanding the role of the phase in the system evolution
    * Using QuTiP to evolve a quantum system in time
  * Playing with different Hamiltonians [10 minutes]
    * Eigenvalues of the Hamiltonian
    * Scaling the energy
    * Changing the differences between energies
  * Break

* Third hour:
  * Recap of where we are [5 minutes]
    * Can code multi-atom states
    * Can can evolve states in time
    * All the ingredients we need to start controlling the states
  * Controlling a quantum system [20 minutes]
    * Time-dependent Hamiltonians
    * Adding driving to the Hamiltonian
    * Visualising the evolution of time-dependent systems with QuTiP
  * Rotating the state of a single atom [15 minutes]
    * Defining the control pulse
    * Solving for the state of the atom as a function of time
    * Visualising the evolution of the state
  * Understanding the Rydberg blockade [10 minutes]
    * Describing the mechanism of interaction between neighbouring atoms
    * Demonstrating how this can be used to implement a two-atom logic gate
  * Break

* Fourth hour:
  * Coupling the system to the environment [15 minutes]
    * Describing the environment
    * Visualising the evolution including decoherence

# 0 minutes

## What are we here for?

What are we going to do in the coming hour? I am going to talk about QuTiP and show you QuTiP being used. Presumably most of you are here to learn about QuTiP, and so presumably you will do whatever is necessary for *you* to learn -- watch, listen, think, make notes, etc.

Two things are vital for learning -- engaging with the material at hand and feedback.

The engaging we have partly covered already, but it is worth emphasising. If you don't engage you won't learn. If you immerse yourself more deeply you will learn more. If you are not going to engage, go do something else. Don't waste your time if you're not going to engage. You are in charge of your own time and your own learning.

Now for feedback.

Inevitably during the learning process we will go astray. I will misexplain something, or explain something in a way that is not right for *you* as you are right now. You will misunderstand things, or fail to see that there is something to understand. None of these are things to be ashamed of -- they are just part of the process.

Effective learning requires detecting and correcting these glitches. In this lecture our primary means of feedback will be questions. Ask questions to check your understanding, and ask immediately if you do not understand something. If you don't understand something, it will make the next thing harder to follow, and you will quickly become lost.

An hour is short, so we will not get to learn very much in it -- only see a few things in one corner of QuTiP -- but if you learn often and regularly, the small pieces of learning add up, until one day someone will ask you in amazement "how did you know how to do that" and you will not be able to say exactly, except to answer "I kepted learning every day for a long time".

Onwards.

## QuTiP and open quantum systems

QuTiP is a Python package for simulating *open quantum systems*.

An open quantum system is a quantum *system* *interacting* within an *environment*, like so:

![Open Quantum System](images/open-qm-sys.png "Open Quantum System")

Reference: [doi:10.1063/1.5115323](https://doi.org/10.1063/1.5115323)

The system evolves according to a Schrödinger equation:

$$
    i \hbar \frac{d \psi}{dt} = H(t) \psi
$$

where $\Psi \in \mathcal{H}$.

Or if the system is represented by a density matrix (i.e. an essemble) then by the von Neumann equation:

$$
    \frac{\partial \rho}{\partial t} = \frac{1}{i \hbar} [H(t),\rho]
$$

where $ \rho $ is an operator $ \rho : \mathcal{H} \rightarrow \mathcal{H} $ where $ \rho = \sum_j p_j \lvert \psi_j \rangle \langle \psi_j \rvert $ and the $ p_j $ are real, positive (or zero) and sum to $ 1 $.

Practicalities:

* The system Hilbert space is tractably small (e.g. a few tens of qubits).
* The environment Hilbert space is intractably large (e.g. a small antenna and its surrounding electromagnetic field)
* The environment is coupled to the system via an interaction term, $ H_I $ in the total system Hamiltonian, $ H_T $.
* The total system evolves unitarily according to the Schrödinger or Von Neuman equation with $ H_T = H_S + H_I + H_E $.
* The environment and its effect on the system must be approximated in some way.

Approximating the effects of the environment:

* Typically the system plus environment is modelled by a new Hilbert space, $ \mathcal{H}_A $, and a new evolution equation for it.
* Both $ \mathcal{H}_A $ and the new evolution equation will dependent on the environment.
* In the simplest examples, $ \mathcal{H}_A $, will sometimes *look* exactly like $ H $. Do not be fooled.
* The new evolution equation will be non-unitary.

## A [Fabry-Pérot interferometer](https://en.wikipedia.org/wiki/Fabry%E2%80%93P%C3%A9rot_interferometer) in a thermal bath

i.e. the very "simplest" example.

* The system (i.e. the interferometer) is represented by [quantum harmonic oscillator](https://en.wikipedia.org/wiki/Quantum_harmonic_oscillator) with frequency $ \omega_c $ and Hamiltonian $ H = \hbar \omega ( N + \frac{1}{2} ) $ and $ N = a^{\dagger} a $ (the number of the excitation).

* The bath may emit photons into the interferometer cavity, or absorb photons from it. This raises or lowers the number of excitations in cavity.

As the approximate Hilbert space $ H_A $ we will use $ H $ itself, but the evolution will be non-unitary and will be government by a Lindblad equation:

$$
    \frac{\partial \rho}{\partial t} = \frac{i}{\hbar}[H, \rho] + \sum_{i=1}^{2} \gamma_i \left( L_i^{\dagger} \rho L_i - \frac{1}{2} \left\{ L_{i}^{\dagger} L_i, \rho \right\} \right)
$$

where:

\begin{align}
    L_1 = a, & & \gamma_1 = \frac{\gamma}{2}(n + 1) \\
    L_2 = a^{\dagger}, & & \gamma_2 = \frac{\gamma}{2}n \\
\end{align}

$ L_1 $ represents the possibility of the system emitting a photon into the bath, and $ L_2 $ the possibility of the system absorbing a photon from the bath.

$ \gamma $ is the strength of the coupling between the bath and the system.

$ n $ is the average occupancy of the bath.

The evolution equation is an example of a [Lindblad equation](https://en.wikipedia.org/wiki/Lindbladian). These describe the evolution of systems coupled to Markovian baths (i.e. where the effect of the bath on the system depends only on the current state of the approximated system and not on the complete evolution history).

The operators $ L $ are called Lindblad or jump operators.

The terms $ L_i^{\dagger} \rho L_i - \frac{1}{2} \left\{ L_{i}^{\dagger} L_i, \rho \right\} $ may be represented by a superoperator called the Liouvillian.

The Lindblad equation and Liouvillians form the basis of many more complex approximations which capture richer interactions between a system and an environment, so they are well worth studying in detail. 

QuTiP includes direct support for solving Lindblad master equations via the master equation solver, `mesolve`.

In [1]:
import qutip

# 15 minutes

# 30 minutes

# 45 minutes

# End

In [2]:
qutip.about()


QuTiP: Quantum Toolbox in Python
Copyright (c) QuTiP team 2011 and later.
Current admin team: Alexander Pitchford, Nathan Shammah, Shahnawaz Ahmed, Neill Lambert, Eric Giguère, Boxi Li, Jake Lishman and Simon Cross.
Board members: Daniel Burgarth, Robert Johansson, Anton F. Kockum, Franco Nori and Will Zeng.
Original developers: R. J. Johansson & P. D. Nation.
Previous lead developers: Chris Granade & A. Grimsmo.
Currently developed through wide collaboration. See https://github.com/qutip for details.

QuTiP Version:      4.7.0
Numpy Version:      1.22.3
Scipy Version:      1.8.0
Cython Version:     0.29.28
Matplotlib Version: 3.5.1
Python Version:     3.9.12
Number of CPUs:     8
BLAS Info:          OPENBLAS
OPENMP Installed:   False
INTEL MKL Ext:      False
Platform Info:      Linux (x86_64)
Installation path:  /home/simon/venvs/py3/qutip-venezuela-2022/lib/python3.9/site-packages/qutip
Please cite QuTiP in your publication.
For your convenience a bibtex reference can be easily gen